In [7]:
using JSON3, DataFrames, CSV, Dates, PlotlyJS, Statistics

In [12]:
capacity_factors = CSV.read("Data/Capacity_Factors.csv",DataFrame);
cap_fac_spain = CSV.read("Data/EU_solar.csv",DataFrame,header=3);
cap_fac_spain.time = Dates.DateTime.(cap_fac_spain.time,"yyyy-mm-dd HH:MM:SS");
rename!(cap_fac_spain, "national"=>"solar-ES");
capacity_factors = innerjoin(capacity_factors,cap_fac_spain,on=:time);
rename!(capacity_factors, "time"=>"date_date");
capacity_factors.date_year = Dates.year.(capacity_factors.date_date);
capacity_factors.date_month = Dates.month.(capacity_factors.date_date);
capacity_factors.date_day = Dates.day.(capacity_factors.date_date);
capacity_factors.date_hour = Dates.hour.(capacity_factors.date_date);
columns = names(capacity_factors);

to_keep = columns[.!occursin.("date",columns)];
date_columns = columns[columns.∉ Ref(to_keep)];

cf = Dict()
for col in to_keep
    cf[col] = capacity_factors[!,[col;date_columns]]
end

long_term_data = Dict(); long_term_year = Dict()
for element in cf
    name, df = element
    gdf = groupby(df, [:date_month, :date_year])
    gdf = combine(gdf, name .=> mean => name)
    long_term_data[name] = gdf

    df = gdf;
    gdf = groupby(df, :date_month)
    gdf = combine(gdf, name .=> mean => name)
    long_term_year[name] = gdf
end

monthly_avg = Dict()
for element in cf
    name, df = element
    avgs = []
    for k in 1:12
        avg = mean(df[(df.date_month.==k),name])
        push!(avgs,avg)
    end
    monthly_avg[name] = avgs
end

avg_data = Dict()
for element in cf
    name, df = element
    avg_matrix = []
    for k in 1:12
        yearly_means = []
        for j in 1980:2019
            yearly_mean = mean(df[(df.date_month.==k) .& (df.date_year.==j),name])
            push!(yearly_means,yearly_mean)
        end
        push!(avg_matrix,yearly_means)  
    end
    avg_data[name] = avg_matrix
end

cols = Dict()
for element in avg_data
    name, matrix = element
    best_value = monthly_avg[name]
    df = cf[name]
    tmy_vector = []
    best_years = []
    for k in 1:12
        error = abs.(matrix[k] .- best_value[k])
        best_year_index = findall(x->x==min(error...),error)
        best_year = 1980 + best_year_index[1] - 1
        best_year_df = df[(df.date_month.==k) .& (df.date_year.==best_year),[name,:"date_year","date_month","date_day","date_hour"]]
        if (k==2)
            best_year_df = best_year_df[best_year_df.date_day.<=28,:]
        end
        tmy_vector = [tmy_vector;best_year_df[!,name]]
        push!(best_years,best_year)
    end
    cols[name] = tmy_vector
end

tmy = DataFrame(cols)
CSV.write("Data/TMY.csv",tmy)

"Data/TMY.csv"

In [88]:
cf["offshore"][!,"offshore"]

350640-element Vector{Float64}:
 0.4354
 0.4337
 0.4438
 0.4694
 0.5028
 0.5306
 0.5504
 0.5644
 0.574
 0.5762
 ⋮
 0.115
 0.1382
 0.1682
 0.1984
 0.2265
 0.2562
 0.2778
 0.2959
 0.3153